<a href="https://colab.research.google.com/github/rajagopal17/Text-Analysis-with-Spacy/blob/master/Glove_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### https://medium.com/analytics-vidhya/basics-of-using-pre-trained-glove-vectors-in-python-d38905f356db

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', size=14)
sns.set(style='white')
sns.set(style='whitegrid', color_codes=True)
import csv
import re
from __future__ import unicode_literals
import spacy
from spacy.tokens import doc
nlp=spacy.load('en')
import en_core_web_sm
#nlp=en_core_web_md.load()
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
parser = English()
import string
punctuations=string.punctuation
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split

import gensim,pprint
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)
from gensim.models import Phrases
#from gensim.models.word2vec import sentences
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import corpora, models,similarities
from gensim.parsing.preprocessing import preprocess_documents
from gensim.models import Word2Vec, KeyedVectors 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
!git clone https://github.com/MohammadWasil/Sentiment-Analysis-IMDb-Movie-Review.git

fatal: destination path 'Sentiment-Analysis-IMDb-Movie-Review' already exists and is not an empty directory.


In [3]:
data_train =pd.read_csv('/content/Sentiment-Analysis-IMDb-Movie-Review/labeledTrainData.tsv', delimiter='\t',encoding="utf-8")
#data_test =('/content/Sentiment-Analysis-IMDb-Movie-Review/testData.tsv')
data_file = data_train[['sentiment','review']].copy()
train_ds = data_file.head(15000).copy()
test_ds  = data_file.tail(5000).copy()
train_ds.to_csv('/content/train_ds.csv',index=False)
test_ds.to_csv('/content/test_ds.csv',index=False)
train_ds.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


# Convert column to string, apply nlp pipe lines for lemmatization,ents, etc
  

1.   Convert column to list and feed it back to new data frame
2.   Get frequency and plot on graph



In [ ]:
#Convert each row of reviews column to string and store it in a file

temp_file=data_train['Reviews'].apply(lambda x:str(x))
temp_file=data_train['Reviews'].apply(lambda x:re.sub('[^A-Z 0-9 a-z-]+','',x))
temp_file

In [ ]:
lemma_list=list([token.lemma_ for token in doc if token.is_stop==False] for doc in nlp.pipe(temp_file, n_threads=2,batch_size=1000,disable=['tagger','parser','ner']))


In [ ]:
lemma_names=list([token.text for token in doc if token.pos_=='PROPN'] for doc in nlp.pipe(temp_file,batch_size=1000))

In [ ]:
lemma_person=list([token.text for token in doc if token.ent_type_=='PERSON'] for doc in nlp.pipe(temp_file,batch_size=1000))

In [ ]:
get_names=[]
for x in lemma_person:
  for y in x:
    get_names.append(y)
get_names

In [ ]:
data_clean['lemma']=lemma_list
data_clean['Reviews']=lemma_names
data_clean['names']=lemma_person

In [ ]:
data_clean.head(20)

In [ ]:
from collections import Counter
word_freq=Counter(get_names)
word_freq



In [ ]:
word_freq_graph =pd.DataFrame(list(word_freq.items()),columns=['name','freq'])
word_freq_graph
final_df=word_freq_graph[word_freq_graph['freq']>800 ]
final_df

In [ ]:
import matplotlib.pyplot as plot
final_df.plot.barh(x='name', y='freq', title="Frequency of the mention of lead character's name");

plot.show(block=True);

# Torch Text

In [ ]:
from torchtext.data import Field,TabularDataset,BucketIterator

In [ ]:
#X_train,X_test,y_train,y_test = train_test_split('/content/Sentiment-Analysis-IMDb-Movie-Review/labeledTrainData.tsv')

In [ ]:
!git clone https://github.com/AladdinPerzon/Machine-Learning-Collection.git

Cloning into 'Machine-Learning-Collection'...
remote: Enumerating objects: 9466, done.
remote: Total 9466 (delta 0), reused 0 (delta 0), pack-reused 9466
Receiving objects: 100% (9466/9466), 1.07 GiB | 39.99 MiB/s, done.
Resolving deltas: 100% (661/661), done.
Checking out files: 100% (154/154), done.


In [ ]:
def tokenize(text):
  return[token.text for token in nlp(text)]

In [ ]:
#tokenize= lambda x: x.split()
TEXT  = Field(sequential= True,use_vocab=True,tokenize=tokenize,lower=True)
LABEL = Field(sequential=False,use_vocab=False)
fields={'sentiment':LABEL,'review':TEXT}

#Split dataset for train & test

In [ ]:
#train_data, test_data= TabularDataset(path='/content/sample_data/test_d.csv',format='csv',fields={'sentiment':sentiment,'review':review})
train = TabularDataset(path='/content/sample_data/train_ds.csv', 
                        format='csv', 
                        fields=[("sentiment",LABEL),
                                ("review",TEXT)],  
                        skip_header=True)
test = TabularDataset(path='/content/sample_data/test_ds.csv', 
                        format='csv', 
                        fields= [("sentiment",LABEL),
                                ("review",TEXT)],  
                        skip_header=True)                             
                       

In [ ]:
TEXT.build_vocab(train,max_size=10000,min_freq=2)

In [ ]:
TEXT.vocab.itos

#TEXT.vocab.stoi['movie']

['<unk>',
 '<pad>',
 'the',
 ',',
 '.',
 'and',
 'a',
 'of',
 'to',
 'is',
 'in',
 'it',
 'i',
 'that',
 'this',
 "'s",
 '-',
 'was',
 '/><br',
 'as',
 'with',
 '"',
 'for',
 'but',
 'movie',
 'film',
 ')',
 'on',
 '(',
 'you',
 'are',
 "n't",
 'not',
 'his',
 'he',
 'have',
 'be',
 'one',
 'all',
 'at',
 'by',
 'an',
 'who',
 'they',
 'from',
 'like',
 'her',
 '!',
 'so',
 "'",
 'about',
 'there',
 'or',
 'has',
 'just',
 'out',
 'good',
 'some',
 'do',
 'if',
 'what',
 'she',
 'more',
 'very',
 'when',
 '?',
 'up',
 'would',
 'no',
 'even',
 'their',
 'time',
 'my',
 'can',
 'which',
 'only',
 'had',
 'really',
 'were',
 'story',
 'did',
 'see',
 'people',
 ':',
 'me',
 'does',
 'than',
 'we',
 'could',
 'well',
 'been',
 'will',
 'get',
 'into',
 'most',
 'much',
 'great',
 'other',
 'because',
 'him',
 'how',
 'first',
 '...',
 'bad',
 'also',
 'then',
 'them',
 'its',
 'made',
 'make',
 'any',
 'way',
 'after',
 'too',
 '/>the',
 'movies',
 'think',
 'characters',
 '<',
 'br',
 'f

In [ ]:
train_ds.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


# Glove Embeddings

In [4]:
X=train_ds[['review']].copy()
y=train_ds[['sentiment']].copy()

Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,train_size=0.7,random_state=32,shuffle=True)

In [5]:
lines=Xtest['review'].apply(lambda x: str(x))

lines_list=list([token.lemma_.lower() for token in doc if token.is_alpha] for doc in nlp.pipe(lines, batch_size=1000,disable=['tagger','parser','ner']))

#lines_list

# How to use standard Glove vectors for finding similarities without Gensim

To load the pre-trained vectors, we must first create a dictionary that will hold the mappings between words, and the embedding vectors of those words.

In [ ]:
import os
import numpy as np
GLOVE_DIR ='/content/drive/My Drive/Python'
print('Indexing word vectors.')
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'),encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

#print(embeddings_index['banana'])

In [79]:
from scipy import spatial
def find_closest_embeddings(embedding):
    return sorted(embeddings_index.keys(), key=lambda word: spatial.distance.euclidean(embeddings_index[word], embedding))


In [80]:
find_closest_embeddings(embeddings_index["japanese"])[:10]

['japanese',
 'japan',
 'tokyo',
 'korean',
 'chinese',
 'taiwanese',
 'kyodo',
 'hashimoto',
 'moreover',
 'meanwhile']

# Converting  Standard Glove vectors into Gensim- Word2Vec format for finding similarities

In [6]:
!pip3 install glove_python
#https://medium.com/analytics-vidhya/word-vectorization-using-glove-76919685ee0b
#https://medium.com/analytics-vidhya/basics-of-using-pre-trained-glove-vectors-in-python-d38905f356db

In [7]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file='/content/drive/My Drive/Python/glove.6B.300d.txt', word2vec_output_file="/content/drive/My Drive/Python/gensim_glove_vectors.txt")    
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format("/content/drive/My Drive/Python/gensim_glove_vectors.txt", binary=False)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-08-06 07:38:51,767 : INFO : converting 400001 vectors from /content/drive/My Drive/Python/glove.6B.300d.txt to /content/drive/My Drive/Python/gensim_glove_vectors.txt
2020-08-06 07:38:58,449 : INFO : loading projection weights from /content/drive/My Drive/Python/gensim_glove_vectors.txt
2020-08-06 07:40:56,177 : INFO : loaded (400001, 300) matrix from /content/drive/My Drive/Python/gensim_glove_vectors.txt


In [74]:

model.most_similar('japanese')



/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('japan', 0.769140362739563),
 ('tokyo', 0.6243131160736084),
 ('korean', 0.6134214401245117),
 ('chinese', 0.5663821697235107),
 ('yen', 0.4867936968803406),
 ('taiwanese', 0.4863933324813843),
 ('hashimoto', 0.46149420738220215),
 ('asian', 0.45883551239967346),
 ('kyodo', 0.45841461420059204),
 ('foreign', 0.4445464611053467)]

# Train Glove model on own corpus, find similarities and save the model

https://github.com/alexeygrigorev/avito-duplicates-kaggle/blob/master/prepare_glove_model.py

In [20]:
from time import time
from glove import Glove, Corpus

def train_glove(sentences):
    print ('training glove model...')
    t0 = time()
    
    num_features = 300    # Word vector dimensionality
    context = 5          # Context window size
    learning_rate = 0.05
    
    corpus = Corpus()
    corpus.fit(sentences, window=context)

    glove = Glove(no_components=num_features, learning_rate=learning_rate)
    glove.fit(corpus.matrix, epochs=30, no_threads=8, verbose=True)
    glove.add_dictionary(corpus.dictionary)

    print('took %0.5fs.' % (time() - t0))
    return (glove)

In [21]:
model=train_glove(lines_list)

training glove model...
Performing 30 training epochs with 8 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
took 169.86294s.


In [15]:
model.most_similar('lead',number=6)

[('audience', 0.9988545790755441),
 ('set', 0.998696491129534),
 ('director', 0.9986048814349453),
 ('eye', 0.9984169486492076),
 ('include', 0.9983186282532661)]

In [17]:
from gensim.models import Word2Vec, KeyedVectors   

model.save('/content/drive/My Drive/Python/glove_final.model')

In [18]:
#from gensim.models import Word2Vec, KeyedVectors 
rmodel=model.load('/content/drive/My Drive/Python/glove_final.model')

In [34]:
rmodel.most_similar('role',number=6)

[('set', 0.9985247659530221),
 ('child', 0.9984764780995089),
 ('family', 0.998461097913626),
 ('in', 0.9981711476296137),
 ('young', 0.9981509456178514)]

# Embedding using Gensim-Word2Vec on own corpus
  Save the model and reuse the saved model

In [142]:
w2v_model=gensim.models.Word2Vec(lines_list,size=50,sg=5,min_count=2,workers=3)

2020-08-06 16:40:03,263 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2020-08-06 16:40:03,266 : INFO : collecting all words and their counts
2020-08-06 16:40:03,268 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-06 16:40:03,455 : INFO : collected 28814 word types from a corpus of 1031321 raw words and 4500 sentences
2020-08-06 16:40:03,456 : INFO : Loading a fresh vocabulary
2020-08-06 16:40:03,502 : INFO : effective_min_count=2 retains 17223 unique words (59% of original 28814, drops 11591)
2020-08-06 16:40:03,503 : INFO : effective_min_count=2 leaves 1019730 word corpus (98% of original 1031321, drops 11591)
2020-08-06 16:40:03,564 : INFO : deleting the raw counts dictionary of 28814 items
2020-08-06 16:40:03,566 : INFO : sample=0.001 downsamples 49 most-common words
2020-08-06 16:40:03,568 : INFO : downsampling leaves estimated 723978 word corpus (71.0% of prior 1019730)
2020-08-06 16:40:03,629 : INFO : estimat

In [143]:
w2v_model.train(lines_list, total_examples=len(lines_list), epochs=10)

2020-08-06 16:40:31,114 : WARNING : Effective 'alpha' higher than previous training cycles
2020-08-06 16:40:31,116 : INFO : training model with 3 workers on 17223 vocabulary and 50 features, using sg=5 hs=0 sample=0.001 negative=5 window=5
2020-08-06 16:40:32,132 : INFO : EPOCH 1 - PROGRESS: at 20.40% examples, 144150 words/s, in_qsize 5, out_qsize 0
2020-08-06 16:40:33,148 : INFO : EPOCH 1 - PROGRESS: at 40.69% examples, 145984 words/s, in_qsize 5, out_qsize 0
2020-08-06 16:40:34,179 : INFO : EPOCH 1 - PROGRESS: at 62.60% examples, 148444 words/s, in_qsize 5, out_qsize 0
2020-08-06 16:40:35,227 : INFO : EPOCH 1 - PROGRESS: at 84.47% examples, 149062 words/s, in_qsize 5, out_qsize 0
2020-08-06 16:40:35,835 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-06 16:40:35,852 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-06 16:40:35,901 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-06 16:40:35,902 : INFO : EP

(7240043, 10313210)

In [144]:
w2v_model.wv.most_similar('plot')

2020-08-06 16:41:18,937 : INFO : precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('storyline', 0.8199487924575806),
 ('dialogue', 0.7991106510162354),
 ('convolute', 0.7771885991096497),
 ('premise', 0.7746027708053589),
 ('story', 0.7615073323249817),
 ('nutshell', 0.7545708417892456),
 ('laughably', 0.7541012167930603),
 ('development', 0.7538366317749023),
 ('linear', 0.7509469389915466),
 ('ridiculousness', 0.742781400680542)]

In [145]:

w2v_model.save("/content/drive/My Drive/Python/word2vec.model")

2020-08-06 16:41:20,462 : INFO : saving Word2Vec object under /content/drive/My Drive/Python/word2vec.model, separately None
2020-08-06 16:41:20,464 : INFO : not storing attribute vectors_norm
2020-08-06 16:41:20,466 : INFO : not storing attribute cum_table
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-08-06 16:41:20,607 : INFO : saved /content/drive/My Drive/Python/word2vec.model


In [146]:
#from gensim.models import Word2Vec, KeyedVectors 

loaded_model = Word2Vec.load('/content/drive/My Drive/Python/word2vec.model')

2020-08-06 16:41:20,622 : INFO : loading Word2Vec object from /content/drive/My Drive/Python/word2vec.model
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-08-06 16:41:20,733 : INFO : loading wv recursively from /content/drive/My Drive/Python/word2vec.model.wv.* with mmap=None
2020-08-06 16:41:20,735 : INFO : setting ignored attribute vectors_norm to None
2020-08-06 16:41:20,737 : INFO : loading vocabulary recursively from /content/drive/My Drive/Python/word2vec.model.vocabulary.* with mmap=None
2020-08-06 16:41:20,739 : INFO : loading trainables recursively from /content/drive/My Drive/Python/word2vec.model.trainables.* with mmap=None
2020-08-06 16:41:20,740 : INFO : setting igno

In [147]:
loaded_model.wv.most_similar('role')

2020-08-06 16:41:20,787 : INFO : precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('sellers', 0.7252811789512634),
 ('actress', 0.7105032205581665),
 ('uncredited', 0.7030029892921448),
 ('tierney', 0.6850537061691284),
 ('grant', 0.6818700432777405),
 ('choice', 0.6799255609512329),
 ('hamlet', 0.6768991947174072),
 ('colman', 0.6652531623840332),
 ('actor', 0.6622415781021118),
 ('miscast', 0.6578194499015808)]

In [150]:
sim_words = {words: [item[0] for item in loaded_model.wv.most_similar([words], topn=5)]
                  for words in ['plot', 'movie','play','role','cast','story','history']}

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [151]:
sim_words

{'cast': ['support', 'actor', 'ensemble', 'stormare', 'persuasively'],
 'history': ['finnish', 'beginning', 'breakthrough', 'revolution', 'wwii'],
 'movie': ['film', 'folks', 'dunno', 'it', 'wrestlemania'],
 'play': ['hilt', 'reprise', 'magruder', 'bowie', 'walker'],
 'plot': ['storyline', 'dialogue', 'convolute', 'premise', 'story'],
 'role': ['sellers', 'actress', 'uncredited', 'tierney', 'grant'],
 'story': ['tale', 'premise', 'plot', 'retell', 'linear']}

In [55]:
loaded_model.wv.vocab.keys()

dict_keys(['well', 'what', 'can', 'it', 'be', 'say', 'about', 'this', 'disaster', 'i', 'watch', 'because', 'air', 'on', 'cable', 'regret', 'for', 'waste', 'my', 'time', 'but', 'at', 'little', 'do', 'money', 'br', 'creature', 'the', 'cheesy', 'you', 'get', 'please', 'need', 'to', 'very', 'generous', 'not', 'anger', 'by', 'cheap', 'halloween', 'costume', 'oh', 'good', 'there', 'also', 'display', 'of', 'horrible', 'act', 'f', 'x', 'and', 'dialogue', 'confrontation', 'with', 'unbelievable', 'a', 'much', 'pathetic', 'scene', 'wrong', 'from', 'direct', 'video', 'flick', 'make', 'full', 'moon', 'pictures', 'movie', 'look', 'like', 'one', 'all', 'cost', 'only', 'thing', 'trash', 'sexy', 'woman', '-pron-', 'see', 'prince', 'in', 'matt', 'think', 'he', 'great', 'singe', 'boy', 'probably', 'his', 'performance', 'music', 'that', 's', 'why', 'win', 'oscar', 'or', 'something', 'now', 'have', 'under', 'belt', 'although', 'curse', 'way', 'film', 'just', 'sure', 'no', 'kid', 'room', 'without', 'reveal'

In [172]:
#a=['film', 'plot', 'movie','play','role','cast','story','history']
j=KeyedVectors.load('/content/drive/My Drive/Python/word2vec.model')

print( [x[0] for x in j.most_similar([v])] for v in ['film', 'plot', 'movie','play','role','cast','story','history'])

2020-08-06 17:04:44,918 : INFO : loading Word2VecKeyedVectors object from /content/drive/My Drive/Python/word2vec.model
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-08-06 17:04:45,196 : INFO : loading wv recursively from /content/drive/My Drive/Python/word2vec.model.wv.* with mmap=None
2020-08-06 17:04:45,197 : INFO : setting ignored attribute vectors_norm to None
2020-08-06 17:04:45,197 : INFO : loading vocabulary recursively from /content/drive/My Drive/Python/word2vec.model.vocabulary.* with mmap=None
2020-08-06 17:04:45,199 : INFO : loading trainables recursively from /content/drive/My Drive/Python/word2vec.model.trainables.* with mmap=None
2020-08-06 17:04:45,201 : INFO : 

<generator object <genexpr> at 0x7fb7a3d662b0>


In [125]:
# Get a list of words in the vocabulary
words = loaded_model.wv.vocab.keys()

# Make a dictionary
we_dict = {word:loaded_model.wv.most_similar([word]) for word in words}

In [179]:
print([ x[0] for x in j.wv.most_similar('story')])

['tale', 'premise', 'plot', 'retell', 'linear', 'nutshell', 'simple', 'concept', 'insightful', 'basic']


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
